# MMVCのリポジトリをGoogle Driveに保存する

ver.2022/12/19

MMVC_Trainerのリポジトリと事前学習済みモデルを、Google Driveに保存します。

​

In [ ]:
#@title ## 1 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージとリポジトリを導入します。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
#git-lfs 事前学習済みモデルのダウンロードに使用
!apt install git-lfs
#os ディレクトリの存在確認に使用
import os
#sys セルの強制終了に使用
import sys
print("log: パッケージのインストールが完了しました。\n")

#MMVC_Trainerリポジトリのクローン
print("log: MMVC_Trainerリポジトリの読み込みを開始します。")
#ホームディレクトリに戻る
%cd /content
##ブランチ一覧取得したいのでno-single-branch, 履歴要らないのでmax depth=1
!git clone --depth=1 --no-single-branch https://github.com/isletennos/MMVC_Trainer/
%cd /content/MMVC_Trainer
print("log: MMVC_Trainerリポジトリの読み込みが完了しました。\n")

#branch一覧取得
print("log: MMVC_Trainerリポジトリのブランチの読み込みを開始します。")
##実行ファイルをリポジトリからダウンロードして実行(仮実装)
##元リポジトリにファイルを含めるなどした方が良いかもしれない
!curl -OL https://raw.githubusercontent.com/Srgr0/MMVC_Trainer_Assets/main/git-branch.sh
!bash /content/MMVC_Trainer/git-branch.sh
##作成されたbranches.txtを開いてPythonのリストを作る
with open("/content/MMVC_Trainer/branches.txt",encoding='utf-8') as inp:
  global repo_branches_list #グローバル指定
  repo_branches_list = inp.readline().split(',')
##Typeの確認(listであればok)
print(type(repo_branches_list))
##内容の確認
print(repo_branches_list)
print("log: MMVC_Trainerリポジトリのブランチの読み込みが完了しました。\n")

#事前学習済みモデルのクローン
print("log: 事前学習済みモデルの読み込みを開始します。")
#ホームディレクトリに戻る
%cd /content
#git-lfsを有効化
!git lfs install
##ブランチ一覧取得したいのでno-single-branch, 履歴要らないのでmax depth=1
##この時点ではgit lfs入れない
!GIT_LFS_SKIP_SMUDGE=1 git clone --depth=1 --no-single-branch https://huggingface.co/MMVC/prelearned-model
%cd /content/prelearned-model
print("log: 事前学習済みモデルの読み込みが完了しました。\n")

print("log: 事前学習済みモデルのブランチの読み込みを開始します。")
##実行ファイルをリポジトリからダウンロードして実行(仮実装)
##元リポジトリにファイルを含めるなどした方が良いかもしれない
!curl -OL https://raw.githubusercontent.com/Srgr0/MMVC_Trainer_Assets/main/git-branch.sh
!bash /content/prelearned-model/git-branch.sh
##作成されたbranches.txtを開いてPythonのリストを作る
with open("/content/prelearned-model/branches.txt",encoding='utf-8') as inp:
  global plm_branches_list #グローバル指定
  plm_branches_list = inp.readline().split(',')
##Typeの確認(listであればok)
print(type(plm_branches_list))
##内容の確認
print(plm_branches_list)
print("log: 事前学習済みモデルのブランチの読み込みが完了しました。\n")

​

In [ ]:
#@title ## 2 セットアップ
#@markdown MMVC_Trainerのセットアップを行います。

#@markdown まず、セルを実行してください。UIが表示されます。

#@markdown Colabを使用している場合
#@markdown * PlatformでColabを選択してください。
#@markdown * DirectoryでMMVC_Trainerの保存先となる、Google Driveのマイドライブ以下のパスを指定してください。よく分からない場合は、変更しなくとも構いません。変更しない場合は、マイドライブ直下に保存されます。
#@markdown * Repo-Branch, PLM-Branch, FMD_Methodについては、変更不要です。
#@markdown * 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown ローカルの場合
#@markdown * PlatformでLocalを選択してください。
#@markdown * DirectoryでMMVC_Trainerの保存先となる、ローカルのパスを指定してください。
#@markdown * Repo-Branch, PLM-Branch, FMD_Methodについては、変更不要です。


#---関数---
def mount_googledrive():
  print("log: Google Driveのマウントを開始します。")
  print("「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。")
  time.sleep(2)
  print("info: 少し時間がかかります。このままお待ちください。")
  from google.colab import drive
  drive.mount('/content/drive')
  print("log: Google Driveのマウントが完了しました。\n")

def checkout_repository_branch():
  print("log: MMVC_Trainerリポジトリのチェックアウトを開始します。")
  %cd /content/MMVC_Trainer
  print("branch:", repo_branch)
  !git checkout $repo_branch
  print("log: MMVC_Trainerリポジトリのチェックアウトが完了しました。\n")

def move_repository():
  print("log: MMVC_Trainerリポジトリの移動を開始します。")
  print("log: 数分かかる場合もありますので、気長にお待ちください。")
  print("移動先:", path)
  if os.path.isdir(path):
    print("既に同名のフォルダが存在しています。directoryを変更するか、既存のフォルダの名称を変更してください。")
    sys.exit()
  else:
    !mv "/content/MMVC_Trainer" {path}
    %cd {path}
  print("log: MMVC_Trainerリポジトリの移動が完了しました。\n")

def download_finemodel():
  print("log: 事前学習済みモデルのチェックアウトを開始します。")
  #branch変更
  %cd /content/prelearned-model
  print("branch:", plm_branch)
  !git checkout $plm_branch
  print("log: 事前学習済みモデルのチェックアウトが完了しました。\n")
  print("log: 事前学習済みモデルのダウンロードを開始します。")
  !git lfs fetch
  !git lfs pull
  !mv *.pth {path}/fine_model/
  print("\nlog: 事前学習済みモデルのダウンロードが完了しました。\n")

#---関数終わり---


#---メイン---
def first_view():
  #widgetsの構成
  global platform_input #グローバル指定
  global directory_input #グローバル指定
  global repo_branch_input #グローバル指定
  global plm_branch_input #グローバル指定
  platform_input = widgets.Dropdown(options=["Colab", "Local"], value="Colab", description='Platform:', disabled=False)
  directory_input = widgets.Text(value='/content/drive/MyDrive', placeholder='/content/drive/MyDrive', description='Directory:', disabled=False)
  repo_branch_input = widgets.Dropdown(options=repo_branches_list, value='main', description='Repo-Branch:', disabled=False)
  plm_branch_input = widgets.Dropdown(options=plm_branches_list, value='main', description='PLM-Branch:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(platform_input, directory_input, repo_branch_input, plm_branch_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)


def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global directory #グローバル指定
  directory = directory_input.value
  global platform #グローバル指定
  platform = platform_input.value
  global repo_branch #グローバル指定
  repo_branch = repo_branch_input.value
  global plm_branch #グローバル指定
  plm_branch = plm_branch_input.value
  global path #グローバル指定
  path = directory + "/MMVC_Trainer-" + repo_branch
  #Colabとそれ以外で処理を分ける
  if platform == "Colab":
    mount_googledrive() #Google Driveのマウント
    checkout_repository_branch() #MMVC_Trainerリポジトリのチェックアウト
    move_repository() #MMVC_Trainerリポジトリの移動
    download_finemodel() #事前学習済みモデルのダウンロード
    print("処理が完了しました。\n")
  elif platform == "Local":
    checkout_repository_branch() #MMVC_Trainerリポジトリのチェックアウト
    move_repository() #MMVC_Trainerリポジトリの移動
    download_finemodel() #事前学習済みモデルのダウンロード
    print("処理が完了しました。\n")


#実行
first_view()


​

In [ ]:
#@title ## 3 確認
#@markdown セルを実行して、以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ```
#@markdown Directory: MMVC_Trainer
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown
#@markdown Directory: MMVC_Trainer/fine_model
#@markdown D_(70000などの数字).pth
#@markdown G_(70000などの数字).pth
#@markdown ```

print("Directory:", path)
!ls -1 {path}
print("\n")
print("Directory:", path, "/fine_model")
!ls -1 {path}/fine_model

​

In [ ]:
#@title ## サポート専用

#@markdown **このセルはお問い合わせの際、指示があった場合のみ使用してください。**

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、ユーザーが本ノートブックで入力した情報の他、MMVCディレクトリ以下のファイルの情報などが含まれます。ファイルを共有する前に、必ずご自身で内容をご確認ください。

#@markdown ダウンロードされたzipファイルの取り扱いについては、サポートの指示に従ってください。

#前回のファイルを消す
!rm -rf /mmvc-debug

#ファイルの準備
import datetime
ｊst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')
!mkdir /mmvc-debug
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    #変数名(str)
    print(ev, end=' : ', file=f) 
    #変数の型(変数名がstrとなっているためevalでkeyに直す)
    print(type(eval(ev)), end=' : ', file=f)
    #変数の内容(変数名がstrとなっているためevalでkeyに直す)
    print(eval(ev), file=f)

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {path} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)